In [1]:
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

We follow the procedure outlined in 3.1.2.1 (Experiment 1)

Currently: 

* $n = 12$
* $k = 8$
* $p \in \{1, 2, 4, 8\}$
* training instances = $100$
* evaluation instances = $500$


Step 1: Generation of $\underline{\beta}, \underline{\gamma} \in \mathbb{R}^p$

In [2]:
from benchmark.random_k_sat import RandomKSAT

# Read in 100 randomly generated CNF(12, 8, r_8) instances
num_instances = 100
n = 12
k = 8
ps = [1, 2, 4, 8, 16]

instances = RandomKSAT.from_poisson(n=n, k=k, instances=num_instances, from_file=True, calc_naive=True)

In [5]:
import torch
import h5py
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit
from k_sat.pytorch_solver.pytorch_optimiser import PytorchOptimiser

optimal_params = {}

for p in ps:
	print(f'Training for p = {p}')
	circuit = PytorchCircuit(num_vars=n, layers=p)
	adam = torch.optim.Adam(circuit.parameters(), lr=0.1, maximize=True)
	epochs = 40 if p < 5 else 60
	optimiser = PytorchOptimiser(circuit, adam, epochs=epochs)
	optimiser.find_optimal_params(instances.formulas)
	# Save to file
	parent_dir = os.path.dirname(os.getcwd())
	dir = f"{parent_dir}/benchmark/instances/n_{n}"
	with h5py.File(f'{dir}/a_params_{p}.hdf5', 'w') as file:
		file.create_dataset(f'gamma', data=circuit.gamma.detach().clone())
		file.create_dataset(f'beta', data=circuit.beta.detach().clone())

Training for p = 1
Epoch 0, p_succ: 0.0004537361965049058
Epoch 10, p_succ: 0.005027129780501127
Epoch 20, p_succ: 0.005658457055687904
Epoch 30, p_succ: 0.005676239263266325
Epoch 40, p_succ: 0.0056748115457594395
Training for p = 2
Epoch 0, p_succ: 0.00046280468814074993
Epoch 10, p_succ: 0.013177448883652687
Epoch 20, p_succ: 0.015451536513864994
Epoch 30, p_succ: 0.01600099727511406
Epoch 40, p_succ: 0.01601913571357727
Training for p = 4
Epoch 0, p_succ: 0.0004948906134814024
Epoch 10, p_succ: 0.03825230523943901
Epoch 20, p_succ: 0.045633066445589066
Epoch 30, p_succ: 0.04776782542467117
Epoch 40, p_succ: 0.04776553064584732
Training for p = 8
Epoch 0, p_succ: 0.0006186074460856616
Epoch 10, p_succ: 0.1140500158071518
Epoch 20, p_succ: 0.12178404629230499
Epoch 30, p_succ: 0.12772956490516663
Epoch 40, p_succ: 0.12977877259254456
Epoch 50, p_succ: 0.13073483109474182
Epoch 60, p_succ: 0.13094967603683472
Training for p = 16
Epoch 0, p_succ: 0.0011520512634888291
Epoch 10, p_succ:

: 

: 

Step 2: Evaluate on satisfiable CNF instances

In [3]:
import h5py
import torch

# Read params in from file
parent_dir = os.path.dirname(os.getcwd())
dir = f"{parent_dir}/benchmark/instances/n_{n}"
optimal_params_r = {}
for p in ps:
	with h5py.File(f'{dir}/a_params_{p}.hdf5', 'r') as file:
		gamma = torch.from_numpy(file.get(f'gamma')[:])
		beta = torch.from_numpy(file.get(f'beta')[:])
		optimal_params_r[p] = (gamma, beta)

In [4]:
optimal_params_r

{1: (tensor([-0.4657]), tensor([0.3213])),
 2: (tensor([-0.3621, -0.5446]), tensor([0.3601, 0.2733])),
 4: (tensor([-0.2846, -0.4394, -0.4921, -0.5560]),
  tensor([0.3642, 0.3263, 0.2929, 0.2025])),
 8: (tensor([-0.1909, -0.3126, -0.3759, -0.4137, -0.4431, -0.4739, -0.4999, -0.5546]),
  tensor([0.3661, 0.3323, 0.3105, 0.2902, 0.2797, 0.2555, 0.2183, 0.1477])),
 16: (tensor([-0.1272, -0.2225, -0.2826, -0.3175, -0.3385, -0.3574, -0.3688, -0.3774,
          -0.3867, -0.4031, -0.4206, -0.4336, -0.4537, -0.4811, -0.5136, -0.5628]),
  tensor([0.3911, 0.3507, 0.3189, 0.2975, 0.2847, 0.2740, 0.2664, 0.2576, 0.2513,
          0.2488, 0.2386, 0.2257, 0.2045, 0.1811, 0.1419, 0.0884]))}

In [7]:
from k_sat.pytorch_solver.pytorch_circuit import PytorchCircuit

k_e = k
n_e = [12, 13] #[i in range(12, 21)]
num_instances_e = 1000

p_succ = {n : {} for n in n_e}

for n in n_e:
	print(f'evaluating n = {n}')

	# Read in random problems
	instances_e = RandomKSAT.from_poisson(n=n, k=k, instances=num_instances_e, from_file=num_instances, calc_naive=True)

	for (p, params) in optimal_params_r.items():
		print(f'evaluating p = {p}')

		# Initialise QAOA circuit
		circuit = PytorchCircuit(n, p, params[0], params[1])

		# Evolve each instance and take mean of p_succ
		prob = 0
		for instance in instances_e.formulas:
			prob += circuit(instance.naive_counts, instance.naive_sats).item()
		
		p_succ[n][p] = prob/num_instances_e

evaluating n = 12
evaluating p = 1
evaluating p = 2
evaluating p = 4
evaluating p = 8
evaluating p = 16
evaluating n = 13
evaluating p = 1
evaluating p = 2
evaluating p = 4
evaluating p = 8
evaluating p = 16


In [8]:
p_succ

{12: {1: 0.005663965009327512,
  2: 0.016110116937546992,
  4: 0.047392316383309666,
  8: 0.12955711629998404,
  16: 0.2895323381918715},
 13: {1: 0.003642761912604328,
  2: 0.011072542423382402,
  4: 0.03519029043079354,
  8: 0.10062992031819885,
  16: 0.23709505256754346}}

In [9]:
import json
parent_dir = os.path.dirname(os.getcwd())
dir = f"{parent_dir}/benchmark/instances"
with open(f'{dir}/p_succ.json', 'w') as f:
    # Write the dictionary to the file as a JSON object with indentation
    json.dump(p_succ, f)
